In [141]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import urllib
import re
import pymongo
import googletrans
from googletrans import Translator

## Scraping

In [142]:
def get_html(url):
    result = requests.get(url)
    return BeautifulSoup(result.text, "html.parser"), result.text

def download_image(url, file_path, file_name):
    full_path = file_path + file_name + '.jpg'
    urllib.request.urlretrieve(url, full_path)

In [143]:
def parse_listing(url="https://www.boot24.com/boot/1976-ketch-baarda-zeegrundel-8222682/"):
    html, source = get_html(url)

    details = html.find("tbody", class_="datatable-section")
    description = html.find("div", class_="detail-description").text

    attributes = {}

    for row in details.find_all("tr"):
        attribute_name = row.find(class_="datatable-title").text
        attribute_value = row.find(class_="datatable-value").text

        attributes[attribute_name] = attribute_value

    
    return description, attributes

In [166]:
html, source = get_html("https://www.boot24.com/boote/typ-segelboote/seite-2/")

In [169]:
id_ = html.find(class_="main-link")["data-reporting-click-product-id"]
id_

'8426422'

In [171]:
img_url = re.findall(r"((https://images.boatsgroup\.com/resize/\d/\d+/\d+/" + str(id_) + r"_.+?)(jpg|png))", source)
img_url[0][0]

'https://images.boatsgroup.com/resize/1/64/22/8426422_0_020820220758_1.jpg'

In [163]:
page = 1
# id_ = 1
listings = []

while True:
    url = f"https://www.boot24.com/boote/typ-segelboote/seite-{page}/"
    page_html, page_source = get_html(url)

    if page_html.find(class_="disabled next"):
        print("Finished scraping")
        break

    print(f"Scraping page {page}")

    for listing in page_html.find_all("li", class_="listing-result listing-result-row"):
        name_ = listing.find(class_="name").text
        price = listing.find(class_="price").text
        location = listing.find(class_="location").text
        offered_by = listing.find(class_="offered-by").text
        link = listing.find(class_="main-link")["href"]
        id_ = listing.find(class_="main-link")["data-reporting-click-product-id"]
        # img_url = re.findall(r"https://images.boatsgroup\.com/resize.+?" + str(id_) + r".+?\.jpg", page_source)[0]
        try:
            img_url = re.findall(r"((https://images.boatsgroup\.com/resize/\d/\d+/\d+/" + str(id_) + r"_.+?)(jpg|png))", source)[0][0]
        except:
            img_url = ""

        listing_url = f"https://www.boot24.com{link}"

        # listing_html = get_html(listing_url)
        # try:
        #     img_url = listing.find("img")["src"]
        if img_url != "":

            download_image(img_url, "images\\", str(id_))
        # except:
        #     pass

        description, attributes = parse_listing(listing_url)


        listings.append({
            "name":name_, 
            "price":price, 
            "location":location, 
            "offerer":offered_by, 
            "link":link, 
            "id":id_, 
            "img_url":img_url, 
            "listing_url":listing_url, 
            "description":description} 
            | attributes
        )

        # id_ +=1

    page += 1

Scraping page 61
Scraping page 62
Scraping page 63
Scraping page 64
Scraping page 65
Scraping page 66
Scraping page 67
Scraping page 68
Scraping page 69
Finished scraping


In [165]:
x = pd.DataFrame(listings)

## Data Preparation

In [160]:
import translators as ts

res = []

for item in list(x.columns):
    res .append(ts.google(item, from_language="auto", to_language="en"))